In [1]:
# import statements
import numpy as np
from path import Planner

In [2]:
maps = {} # dictionary of maps, a map is a dictionary of coordinates and their corresponding values - one map for each layer

# instantiate map
def instantiate_map():
    map = {}
    for i in np.arange(0, 379.4, 0.1):
        for j in np.arange(0, 189.7, 0.1):
            map[(round(i, 1), round(j, 1))] = None
    return map

In [3]:
arm_1_cardinal_maps = {}
arm_2_cardinal_maps = {}

In [4]:
# linear interpolation
# returns a list of points between two points

def get_points(x1, x2, y1, y2, step_size = 0.1):
    if x1 == x2:
        y_points = np.round(np.append(np.arange(y1, y2, step_size, dtype='float64'), y2), 1)
        x_points = np.round(np.full((np.size(y_points),), x1), 1)

        assert np.size(x_points) == np.size(y_points)
        return x_points, y_points
    
    x_points = np.round(np.append(np.arange(x1, x2, step_size, dtype='float64'), x2), 1)
    y_points = np.zeros(np.size(x_points), dtype='float64')
    
    for i in range(np.size(x_points)):
        y_points[i] = round(((y2-y1)/(x2-x1))*(x_points[i]-x1) + y1, 1)
    
    assert np.size(x_points) == np.size(y_points)
    return x_points, y_points

In [5]:
def update_all_maps():
    path = Planner('sample_gcode/test_big_box.gcode')
    x_start = y_start = None # start point of the path set to None
    
    count = 0
    for coord in path.generate():
        x, y, z, command = coord
                
        # round coordinates to 1 decimal place
        x = round(x, 1)
        y = round(y, 1)
        
        # move coordinates to the left from (0, 0)
        x = x - 5.2
        y = y - 5.2
        
        if z not in maps:
            # create a new map for this z value
            print("Working on layer: ", z)
            maps[z] = instantiate_map()
            arm_1_cardinal_maps[z] = {}
            arm_2_cardinal_maps[z] = {}
            x_start = y_start = None
            count = 0
            
        # for the first point in the path
        if x_start is None and y_start is None:
            x_start = x
            y_start = y
        
        # linear interpolation of points between the start and end points
        x_points, y_points = get_points(x_start, x, y_start, y)
        
        # update the start point
        x_start = x
        y_start = y
        
        # update the map for both arms
        for x_point, y_point in zip(x_points, y_points):
            x_point = round(x_point, 1)
            y_point = round(y_point, 1)
            x_point_2 = round(x_point + 189.7, 1)
            maps[z][(x_point, y_point)] = ("arm_1", command)
            maps[z][(x_point_2, y_point)] = ("arm_2", command)
            arm_1_cardinal_maps[z][count] = (x_point, y_point)
            arm_2_cardinal_maps[z][count] = (x_point_2, y_point)
            count+=1            

In [6]:
# clear all maps
def clear_maps():
    maps.clear()

In [7]:
def clear_cardinal_maps():
    arm_1_cardinal_maps.clear()
    arm_2_cardinal_maps.clear()

In [8]:
# clear all maps and update them
clear_maps()
clear_cardinal_maps()
update_all_maps()

Working on layer:  0.3
Working on layer:  0.45
Working on layer:  0.6
Working on layer:  0.75
Working on layer:  0.9
Working on layer:  1.05
Working on layer:  1.2
Working on layer:  1.35
Working on layer:  1.5
Working on layer:  1.65
Working on layer:  1.8
Working on layer:  1.95
Working on layer:  2.1
Working on layer:  2.25
Working on layer:  2.4
Working on layer:  2.55
Working on layer:  2.7
Working on layer:  2.85
Working on layer:  3


In [12]:
layers = [0.3, 0.45, 0.6, 0.75, 0.9, 1.05, 1.2, 1.35, 1.5, 1.65, 1.8, 1.95, 2.1, 2.25, 2.4, 2.55, 2.7, 2.85, 3.0]

for layer in layers:
    print(f"Length of arm1_cardinal_map at layer {layer} is {len(arm_1_cardinal_maps[layer])}")
    print(f"Length of arm2_cardinal_map at layer {layer} is {len(arm_2_cardinal_maps[layer])}")

for layer in layers:
    for key, value in arm_1_cardinal_maps[layer].items():
        assert maps[layer][value] is not None

    for key, value in arm_1_cardinal_maps[layer].items():
        assert maps[layer][value] is not None
    

Length of arm1_cardinal_map at layer 0.3 is 325917
Length of arm2_cardinal_map at layer 0.3 is 325917
Length of arm1_cardinal_map at layer 0.45 is 325850
Length of arm2_cardinal_map at layer 0.45 is 325850
Length of arm1_cardinal_map at layer 0.6 is 327801
Length of arm2_cardinal_map at layer 0.6 is 327801
Length of arm1_cardinal_map at layer 0.75 is 325850
Length of arm2_cardinal_map at layer 0.75 is 325850
Length of arm1_cardinal_map at layer 0.9 is 327801
Length of arm2_cardinal_map at layer 0.9 is 327801
Length of arm1_cardinal_map at layer 1.05 is 325850
Length of arm2_cardinal_map at layer 1.05 is 325850
Length of arm1_cardinal_map at layer 1.2 is 43904
Length of arm2_cardinal_map at layer 1.2 is 43904
Length of arm1_cardinal_map at layer 1.35 is 43905
Length of arm2_cardinal_map at layer 1.35 is 43905
Length of arm1_cardinal_map at layer 1.5 is 43905
Length of arm2_cardinal_map at layer 1.5 is 43905
Length of arm1_cardinal_map at layer 1.65 is 43905
Length of arm2_cardinal_map a

In [ ]:
# save maps to file
import pickle

count = 0
for map in maps:
    # write dictionary to pkl file
    with open('maps/map_' + str(count) + '.pkl', 'wb') as fp:
        pickle.dump(maps[map], fp)
        print(f"dictionary {count} saved successfully to file")
    count+=1

In [13]:
# save cardinal_maps to file
import pickle

count = 0
for map in arm_1_cardinal_maps:
    # write dictionary to pkl file
    with open('cardinal_maps/arm1_cardinal_map_' + str(count) + '.pkl', 'wb') as fp:
        pickle.dump(arm_1_cardinal_maps[map], fp)
        print(f"dictionary {count} saved successfully to file")
    count+=1
        
count = 0
for map in arm_2_cardinal_maps:
    with open('cardinal_maps/arm2_cardinal_map_' + str(count) + '.pkl', 'wb') as fp:
        pickle.dump(arm_2_cardinal_maps[map], fp)
        print(f"dictionary {count} saved successfully to file")
    count+=1

count = 0
for map in maps:
    with open('maps/map_' + str(count) + '.pkl', 'wb') as fp:
        pickle.dump(maps[map], fp)
        print(f"dictionary {count} saved successfully to file")
    count+=1
    

dictionary 0 saved successfully to file
dictionary 1 saved successfully to file
dictionary 2 saved successfully to file
dictionary 3 saved successfully to file
dictionary 4 saved successfully to file
dictionary 5 saved successfully to file
dictionary 6 saved successfully to file
dictionary 7 saved successfully to file
dictionary 8 saved successfully to file
dictionary 9 saved successfully to file
dictionary 10 saved successfully to file
dictionary 11 saved successfully to file
dictionary 12 saved successfully to file
dictionary 13 saved successfully to file
dictionary 14 saved successfully to file
dictionary 15 saved successfully to file
dictionary 16 saved successfully to file
dictionary 17 saved successfully to file
dictionary 18 saved successfully to file
dictionary 0 saved successfully to file
dictionary 1 saved successfully to file
dictionary 2 saved successfully to file
dictionary 3 saved successfully to file
dictionary 4 saved successfully to file
dictionary 5 saved successfully

In [ ]:
# extract maps from file
import pickle

new_cardinal_maps = {}
count = 0

while True:
    try:
        # read dictionary from pkl file
        with open('cardinal_maps/arm1_cardinal_map_' + str(count) + '.pkl', 'rb') as fp:
            new_cardinal_maps[count] = pickle.load(fp)
            print(f"dictionary {count} loaded successfully from file")
        count += 1
    except FileNotFoundError:
        break

In [ ]:
# extract maps from file
import pickle

maps = {}
count = 0

while True:
    try:
        # read dictionary from pkl file
        with open('maps/map_' + str(count) + '.pkl', 'rb') as fp:
            maps[count] = pickle.load(fp)
            print(f"dictionary {count} loaded successfully from file")
        count += 1
    except FileNotFoundError:
        break